### Importações

In [1]:
import pandas as pd; pd.set_option('display.max_columns', None)
from datetime import date, timedelta
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [2]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [3]:
today = date.today()
tomorrow = today + timedelta(days=1)

day = tomorrow

In [4]:
data_var = day.strftime('%Y-%m-%d')
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
# ligas = temporada[temporada['Date'] == data_var]['League'].unique().tolist()
# temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos.to_csv(f'jogos_processados/{data_var}.csv', index=False)

jogos[['Date', 'League', 'Time', 'Home', 'Away', 
       'Odds_H', 'Odds_A', 'Over_Line', 'Odds_Over', 'Odds_Under', 'HA_Line',
       'HA_Odds_H', 'HA_Odds_A', ]]

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A
6917,2023-11-17,Brasil Nbb,20:00,Caxias do Sul,São José,2.14,1.69,149.5,1.88,1.79,1.5,1.97,1.74
6918,2023-11-17,Brasil Nbb,20:00,Corinthians Paulista,Paulistano,1.73,2.06,146.5,1.88,1.88,-2.5,1.98,1.74
6919,2023-11-17,Brasil Nbb,20:00,Pinheiros,São Paulo,2.32,1.59,157.5,1.86,1.88,2.5,1.97,1.75
6920,2023-11-17,Eua Nba,00:00,Golden State Warriors,Oklahoma City Thunder,2.37,1.69,225.5,1.83,1.93,-4.5,1.90,1.90
6921,2023-11-17,Eua Nba,21:00,Washington Wizards,New York Knicks,3.35,1.40,226.5,1.80,2.01,5.5,2.08,1.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6996,2023-11-17,Romênia Divisão A,14:00,CSM Targu Mures,BCM Pitesti,1.81,1.89,157.5,1.84,1.82,-1.5,1.94,1.77
6997,2023-11-17,Rússia Liga Vtb United,11:00,Uralmash Ekaterinburg,Samara,1.60,2.39,162.5,1.89,1.89,-4.5,2.00,1.76
6998,2023-11-17,Rússia Liga Vtb United,13:00,Nizhny Novgorod,MBA Moscow,1.92,1.90,154.5,1.80,1.95,-1.5,2.02,1.74
6999,2023-11-17,Suíça Liga Sb,15:30,Sdent BBC Nyon,BBC Monthey,1.72,2.07,154.5,1.87,1.79,-2.5,1.95,1.76


## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
8,22:00,Eua Nba,New Orleans Pelicans,Denver Nuggets,222.5,1.81,0.9999,Over
10,16:00,Europa Euroliga,Lyon-Villeurbanne,Baskonia,164.5,1.91,0.7272,Over


### Over v2 (NB)

In [9]:
# Duplicando o dataset
df_over_v2 = jogos.copy()

df_over_v2.reset_index(drop=True, inplace=True)
df_over_v2.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v2.empty:
    mdl_over_v2 = load_model('ML/over/v2/v2_over', verbose=False)
    prev = predict_model(mdl_over_v2, df_over_v2)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo
    prev = prev[((prev['Last_CG_H'] >= 0) & (prev['Last_CG_H'] < 130)) | ((prev['Last_CG_H'] >= 183) & (prev['Last_CG_H'] < 1485))]

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v2_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V2 (NB) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V2 (NB) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
3,00:00,Eua Nba,Golden State Warriors,Oklahoma City Thunder,225.5,1.83,0.6008,Over
5,21:30,Eua Nba,Cleveland Cavaliers,Detroit Pistons,221.5,1.92,0.9768,Over
6,21:30,Eua Nba,San Antonio Spurs,Sacramento Kings,232.5,1.82,0.5504,Over
9,15:00,Europa Euroliga,Zalgiris Kaunas,Bayern,157.5,1.91,0.7278,Over
10,16:00,Europa Euroliga,Lyon-Villeurbanne,Baskonia,164.5,1.91,0.7401,Over
11,16:15,Europa Euroliga,Panathinaikos,Virtus Bologna,161.5,1.87,0.9189,Over
12,16:30,Europa Euroliga,Barcelona,Valencia,156.5,1.91,0.7532,Over
13,16:45,Europa Euroliga,Partizan,Alba Berlin,168.5,1.91,0.7301,Over
22,21:00,Argentina Liga A,Argentino,Platense,156.5,1.88,0.5336,Over
23,22:00,Argentina Liga A,Quimsa,Obera TC,162.5,1.87,0.5046,Over


### Over v3 (NB)

In [10]:
# Duplicando o dataset
df_over_v3 = jogos.copy()

df_over_v3.reset_index(drop=True, inplace=True)
df_over_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v3.empty:
    mdl_over_v3 = load_model('ML/over/v3/v3_over', verbose=False)
    prev = predict_model(mdl_over_v3, df_over_v3)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v3_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V3 (NB) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V3 (NB) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
5,21:30,Eua Nba,Cleveland Cavaliers,Detroit Pistons,221.5,1.92,0.8152,Over
6,21:30,Eua Nba,San Antonio Spurs,Sacramento Kings,232.5,1.82,0.5336,Over
7,22:00,Eua Nba,Chicago Bulls,Orlando Magic,215.5,1.85,0.6657,Over
9,15:00,Europa Euroliga,Zalgiris Kaunas,Bayern,157.5,1.91,0.7955,Over
10,16:00,Europa Euroliga,Lyon-Villeurbanne,Baskonia,164.5,1.91,0.8064,Over
11,16:15,Europa Euroliga,Panathinaikos,Virtus Bologna,161.5,1.87,0.7774,Over
12,16:30,Europa Euroliga,Barcelona,Valencia,156.5,1.91,0.7954,Over
13,16:45,Europa Euroliga,Partizan,Alba Berlin,168.5,1.91,0.7997,Over
21,20:00,Argentina Liga A,San Lorenzo,Independiente de Oliva,153.5,1.88,0.5008,Over
22,21:00,Argentina Liga A,Argentino,Platense,156.5,1.88,0.5058,Over
